# CVEFixes by language

In [5]:
!nvidia-smi

Thu May  9 10:56:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000000:1A:00.0 Off |                    0 |
| N/A   29C    P0              53W / 300W |   1989MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Retrieve dataset from HF

In [2]:
import pandas as pd
from datasets import load_dataset
import os

os.environ["HF_HOME"] = "/tmp/huggingface"
# dataset = load_dataset("euisuh15/cveFixes1")
dataset = load_dataset("euisuh15/cveFixes1", split='c')

# lang option:  c | go | java | python | ruby
# lang = "c"
# df = pd.DataFrame(dataset[lang])
df = pd.DataFrame(dataset)

df.shape

/home/local/QCRI/ejeong/anaconda2/envs/vllm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(2621, 14)

In [3]:
df.head()

,file_name,method_name,code_before,code_after,func_before,func_after,diff,num_lines_added,num_lines_deleted,num_lines_in_file,num_tokens_in_file,repo,cve_id,cwe_id
0,open_utils.c,read_new_config_info,//////////////////////////////////////////////...,//////////////////////////////////////////////...,static int read_new_config_info (WavpackContex...,static int read_new_config_info (WavpackContex...,"{'added': [(563, ' if (bytecnt >= 2) {'), (...",5.0,1.0,825.0,6305.0,https://github.com/dbry/WavPack,CVE-2016-10169,['CWE-125']
1,route.c,inet_rtm_getroute,/*\n * INET\t\tAn implementation of the TCP/IP...,/*\n * INET\t\tAn implementation of the TCP/IP...,static int inet_rtm_getroute(struct sk_buff *i...,static int inet_rtm_getroute(struct sk_buff *i...,"{'added': [(2766, '\tif (rtm->rtm_flags & RTM_...",9.0,2.0,2327.0,14667.0,https://github.com/torvalds/linux,CVE-2017-13686,['CWE-476']
2,main.c,mp4boxMain,/*\n *\t\t\tGPAC - Multimedia Framework C SDK\...,/*\n *\t\t\tGPAC - Multimedia Framework C SDK\...,"int mp4boxMain(int argc, char **argv)\n{\n\tu3...","int mp4boxMain(int argc, char **argv)\n{\n\tu3...","{'added': [(5598, '\t\t\tfprintf(stderr, ""File...",1.0,1.0,5787.0,43960.0,https://github.com/gpac/gpac,CVE-2020-23930,['CWE-476']
3,udp.c,udpv6_sendmsg,/*\n *\tUDP over IPv6\n *\tLinux INET6 impleme...,/*\n *\tUDP over IPv6\n *\tLinux INET6 impleme...,"int udpv6_sendmsg(struct sock *sk, struct msgh...","int udpv6_sendmsg(struct sock *sk, struct msgh...","{'added': [(1113, '\tstruct ipv6_txoptions *op...",6.0,2.0,1221.0,7876.0,https://github.com/torvalds/linux,CVE-2016-3841,"['CWE-264', 'CWE-416']"
4,api.c,crypto_larval_lookup,/*\n * Scatterlist Cryptographic API.\n *\n * ...,/*\n * Scatterlist Cryptographic API.\n *\n * ...,struct crypto_alg *crypto_larval_lookup(const ...,struct crypto_alg *crypto_larval_lookup(const ...,"{'added': [(219, '\t\trequest_module(""crypto-%...",2.0,2.0,424.0,2423.0,https://github.com/torvalds/linux,CVE-2013-7421,['CWE-269']


In [4]:
df.to_csv('vul7.csv')

## VLLM

In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [ ]:
from vllm import LLM, SamplingParams

sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

llm = LLM(model="meta-llama/Meta-Llama-3-8B-Instruct", dtype="half")

before_acc = []
after_acc = []

for i in range(df.shape[0]):
    prompts = [
        promptTemplate(instruction, df.func_before.iloc[i], critical_cwe),
        promptTemplate(instruction, df.func_after.iloc[i], critical_cwe),
    ]

    outputs = llm.generate(prompts, sampling_params)

    before_acc.append(outputs[0].outputs[0].text)
    after_acc.append(outputs[1].outputs[0].text)

2024-05-08 11:32:02,465	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


WARNING 05-08 11:32:02 config.py:618] Casting torch.bfloat16 to torch.float16.
INFO 05-08 11:32:02 llm_engine.py:87] Initializing an LLM engine with config: model='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 05-08 11:32:07 weight_utils.py:163] Using model weights format ['*.safetensors']
INFO 05-08 11:32:16 llm_engine.py:357] # GPU blocks: 5731, # CPU blocks: 2048
INFO 05-08 11:32:19 model_runner.py:684] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 05-08 11:32:19 model_runner.py:688] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 05-08 11:32:25 model_runner.py:756] Graph capturing finished in 6 secs.


Processed prompts:   0%|                                                                                                                                     | 0/2 [00:00<?, ?it/s]

...

Processed prompts:   0%|                                                                                                                                     | 0/2 [00:00<?, ?it/s]

WARNING 05-08 11:59:38 scheduler.py:195] Input prompt (14641 tokens) is too long and exceeds limit of 8192
WARNING 05-08 11:59:38 scheduler.py:195] Input prompt (14538 tokens) is too long and exceeds limit of 8192


Processed prompts:   0%|                                                                                                                                     | 0/2 [00:00<?, ?it/s]

WARNING 05-08 12:00:07 scheduler.py:195] Input prompt (11828 tokens) is too long and exceeds limit of 8192
WARNING 05-08 12:00:07 scheduler.py:195] Input prompt (11928 tokens) is too long and exceeds limit of 8192


Processed prompts:   0%|                                                                                                                                     | 0/2 [00:00<?, ?it/s]